In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
y = pd.read_excel('F:/Subtraction/price/选择取数/XXXX20220629.xlsx', sheet_name=1).fillna(-1)
dic1 = {}
for index, value in enumerate(set(y['车型名称'])):
    dic1[value]= index

dic2 = {}
for index, value in enumerate(set(y['compname'])):
    dic2[value]= index

dic4 = {}
for index, value in enumerate(set(y['修理厂类型'])):
    dic4[value]= index

X = np.vstack((y['车型名称'].map(lambda x: dic1.get(x) if dic1.get(x) else -1), 
                   y['修理厂类型'].map(lambda x: dic4.get(x) if dic4.get(x) else -1), 
                   y['compname'].map(lambda x: dic2.get(x) if dic2.get(x) else -1), 
                   y['配件定损'], y['配件核损'], y['配件残值'])).T
# usecols=['（配）所属区间', '（工）所属区间']
Y = y['（配）所属区间'].astype('str')

In [3]:
len(X)

125315

In [4]:
Y

0         -B
1          A
2         -A
3          B
4          A
          ..
125310    -A
125311    -A
125312     A
125313    -A
125314    -A
Name: （配）所属区间, Length: 125315, dtype: object

In [ ]:
# 参数配置
# 随机森林中树的个数
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 200)]
# 每一节点考虑切分的节点数
max_features = ['auto', 'sqrt']
# 最大深度
max_depth = [int(x) for x in np.linspace(5, 100, num = 5)]
max_depth.append(None)
# 切分一个节点最小数量
min_samples_split = [1, 2,]
# 每一叶子节点最小数量
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
 
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# random_grid = {'n_estimators': n_estimators}

forest = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator=forest, param_distributions=random_grid,
                              scoring='neg_mean_absolute_error', n_jobs=4,n_iter = 100,
                               verbose = 2,cv = 3,random_state = 0)
rf_random.fit(X, Y)

In [40]:
cvres = rf_random.cv_results_
# for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
#     print(np.sqrt(-mean_score), params)

print("寻找的结果：", rf_random.best_estimator_)


寻找的结果： RandomForestClassifier(max_depth=28, max_features='sqrt', n_estimators=1360)


In [32]:
rf_random = RandomForestClassifier(n_estimators = 100, max_features = 'sqrt', min_samples_leaf = 1, max_depth = 20)
rf_random.fit(X, Y)

# cvres = rf_random.cv_results_
# for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
#     print(np.sqrt(-mean_score), params)

# print("寻找的结果：", rf_random.best_estimator_)
# multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)

RandomForestClassifier(max_depth=20, max_features='sqrt')

In [41]:
rf_random.predict(X)
result = rf_random.predict(X)

print("正确率：",accuracy_score(Y, result))
print("正确个数：",accuracy_score(Y, result, normalize=False))


正确率： 0.9404939552328133
正确个数： 117858


In [42]:
from sklearn.metrics import confusion_matrix
C=confusion_matrix(Y, result, labels=["-E", "-D", "-C", "-B", "-A", "A", "B", "C", "D", "E"])
print("混淆矩阵：")
print(C)
# y['result1'] = result
# loo = y[y['（配）所属区间'].astype('str') != y['result1'].astype('str')]
# print(loo)

混淆矩阵：
[[  739     0     0     0    28    22     0     0     0     0]
 [    0  2450     0     0    66    42     0     0     0     0]
 [    0     0  2714     0   194    55     0     0     0     0]
 [    0     0     0  2176   369    92     0     0     0     0]
 [    0     0     0     0 62634    40     0     0     0     0]
 [    0     0     0     0  5641 38421     0     0     0     0]
 [    0     0     0     0   254   191  2089     0     0     0]
 [    0     0     0     0   172   117     0  2091     0     0]
 [    0     0     0     0    80    58     0     0  2017     0]
 [    0     0     0     0    22     9     0     0     0  1413]]


In [8]:
X9196_data = pd.read_excel('F:/Subtraction/price/选择取数/XXXX20220629范围.xlsx', sheet_name=0).fillna(-1)
X9196 = np.vstack((X9196_data['车型名称'].map(lambda x: dic1.get(x) if dic1.get(x) else -1), 
                   X9196_data['修理厂类型'].map(lambda x: dic4.get(x) if dic4.get(x) else -1), 
                   X9196_data['compname'].map(lambda x: dic2.get(x) if dic2.get(x) else -1), 
                   X9196_data['配件定损'], X9196_data['配件核损'], X9196_data['配件残值'])).T

In [43]:
result2 = rf_random.predict(X9196)


In [26]:
print("正确率：",accuracy_score(X9196_data['（配）所属区间'], result2))
print("正确个数：",accuracy_score(X9196_data['（配）所属区间'], result2, normalize=False))

正确率： 0.4961784560459591
正确个数： 78941


In [44]:
print("正确率：",accuracy_score(X9196_data['（配）所属区间'], result2))
print("正确个数：",accuracy_score(X9196_data['（配）所属区间'], result2, normalize=False))

正确率： 0.4983909288614565
正确个数： 79293


In [27]:
C2=confusion_matrix(X9196_data['（配）所属区间'], result2, labels=["-E", "-D", "-C", "-B", "-A", "A", "B", "C", "D", "E"])
print("混淆矩阵：")
print(C2)

混淆矩阵：
[[  156   114    13    14    48   123     0     2     1     5]
 [   17  1148   219    45   354  1008     0     0     0     2]
 [    7   188   441    95  1135  1479    10    10     3     2]
 [    4    79   116    58  1461  1406     6     3     4     1]
 [    5    54    90    59 36599 33327    56    26    24     8]
 [    6    15    34    42 28575 39385    64    69    60    11]
 [    3     3     9     6   781  1521    35    63    74    36]
 [    3     5     4     2   544  1808    46   111   163    64]
 [    6     2    10     5   333  1479    36   162   381   175]
 [    9     2     1     1   108  1018    23   126   321   627]]


In [14]:
# fr = open("test.txt",'r+')
# 读取的str转换为字典
# dic = eval(fr.read()) 
# print(dic)
# fr.close()
fw1 = open("相同车型dic1.txt", 'w+', encoding='utf-8')
# 把字典转化为str
fw1.write(str(dic1))      
fw1.close()
fw2 = open("相同车型dic2.txt", 'w+', encoding='utf-8')
fw2.write(str(dic2))
fw2.close()
fw4 = open("相同车型dic4.txt", 'w+', encoding='utf-8')
fw4.write(str(dic4))
fw4.close()

In [16]:
import joblib
# 存储
joblib.dump(rf_random, "1.4w配件训练（相同车型）.m") 
# 调用
# clf = joblib.load("train_model.m") 

['1.4w配件训练（相同车型）.m']

In [17]:
X9196_data['预测结果'] = result2